In [1]:
import sys; sys.path.append(_dh[0].split("knowknow")[0])
from knowknow import *

**instructions** carefully set the user parameters and run the entire notebook

**output** The variable is called `"<database_name>.<dtype>.ysum"`. For example, `"sociology-wos.ta.ysum"`

**description** The following notebook performs analyses on the time trend of each "item" of type `dtype` (`"ta"` for cited author, `"c"` for cited work, or `"t"` for cited term).
These statistics are universally useful, and computing them all once saves code and computation time in later analyses.
It also standardizes common operations, reducing the probability of a bug going unnoticed.

# User parameters

`database_name` You should already have a database of counts to use this notebook, e.g. `"sociology-wos"`. 

`dtype` The item of interest (e.g. `"ta"` for cited author, `"c"` for cited work, or `"t"` for cited term)

In [2]:
database_name = "sociology-wos"
dtype = 'c'

In [3]:
# Parameters
database_name = "sociology-jstor"
dtype = "t"


In [3]:
pubyears = None

if 'wos' in database_name and 'jstor' in database_name:
    raise Exception("Please put 'wos' or 'jstor' but not both in any database_name.")
    
elif 'wos' in database_name:
    pubyears = load_variable("%s.pubyears" % database_name)
    print("Pubyears loaded for %s entries" % len(pubyears.keys()))
    RELIABLE_DATA_ENDS_HERE = 2019
    database_type = 'wos'
    
elif 'jstor' in database_name:
    RELIABLE_DATA_ENDS_HERE = 2010
    database_type = 'jstor'
    
else:
    raise Exception("Please include either 'wos' or 'jstor' in the name of the variable. This keys which data processing algorithm you used.")

Pubyears loaded for 397702 entries


# Helper class

In [4]:
class ysums:
    meta = defaultdict(int)
    def __init__(self):
        self.ysums = []
    def add(self, x):
        self.ysums.append(x)

In [5]:
class ysum:
    init_stack = []
    
    def __init__(self, **kwargs):        
        self.dict_filter = set(list(self.__dict__)+["dict_filter"]) # save dict keys
        
        for k,v in kwargs.items():
            setattr(self,k,v)
    
        for name in self.init_stack:
            getattr(self,name)()
                
    def dump(self):
        to_return_keys = set(self.__dict__).difference(self.dict_filter)
        
        return {
            k: getattr(self,k)
            for k in to_return_keys
            if k[0] != '_'
        }
    
    @classmethod
    def addinit(cls, st):
        if st not in cls.init_stack:
            cls.init_stack.append(st)

# Getting data

In [6]:
ysum.cits = get_cnt("%s.doc"%database_name, [comb(dtype,'fy'),'fy',dtype])

Loaded keys: dict_keys(['c.fy', 'fy', 'c'])
Available keys: ['c', 'c.fj', 'c.fy', 'c.fy.j', 'fa', 'fa.c', 'fa.fj', 'fa.fj.fy', 'fa.fy', 'fj', 'fj.fy', 'fj.ta', 'fj.ty', 'fy', 'fy.ta', 'fy.ty', 'ta', 'ty']


The following reorganizes the data slightly, for efficiency

In [7]:
ysum.cy = defaultdict(lambda:defaultdict(int))

for cross,count in ysum.cits[ comb(dtype, 'fy') ].items():
    ysum.cy[ getattr(cross, dtype) ][ cross.fy ] = count

# computing simple and universal statistics

In [8]:
ysum.addinit("simple_stats")

class ysum(ysum):
    
    def simple_stats(self):
        self._c = self.cy[ self.name ]
        self._p =  {
            y: count / self.cits['fy'][(y,)]
            for y,count in self._c.items()
            if count > 0 and self.cits['fy'][(y,)] > 0
        }
        
        self.first = min(self._c)
        self.last = max(self._c)
        self.maxcounty = max(self._c, key=lambda y:(self._c[y],y))
        self.maxpropy = max(self._p, key=lambda y:(self._p[y],y))

        self.maxprop = self._p[ self.maxpropy ]
        self.maxcount = self._c[ self.maxcounty ]
        self.total = sum(self._c.values())
        self.totalprop = sum(self._p.values())
        
        if dtype == 'c':
            # extracts some extra information from the name
            
            self.type = 'article'
            if database_type == 'wos':
                sp = self.name.split("|")
                if len(sp) < 2:
                    raise Exception(self.name)
                try:
                    self.pub = int(sp[1])
                    self.type = 'article'
                except ValueError:
                    self.type = 'book'
                    self.pub = pubyears[self.name]

            elif database_type == 'jstor':
                inparens = re.findall(r'\(([^)]+)\)', self.name)[0]
                self.pub = int(inparens)
    
            
    def sum_between(self, A, B): #not including B
        if A > B:
            raise Exception("Can only sum forwards in time. Fix your code...")
        return sum( c for y,c in self._c.items() if B > y >= A )


# definitions of death

death$_1$ is the last year of maximum citations, 
    as long as it's before `RELIABLE_DATA_ENDS_HERE`

In [9]:
#ysum.addinit("_death1")

class ysum(ysum):
    def _death1(self):
        # DEFINING DEATH1
        # death1 is max, as long as it's before RELIABLE_DATA_ENDS_HERE
        if self.maxpropy <= RELIABLE_DATA_ENDS_HERE:
            self.death1 = self.maxcounty
            return
        
        self.death1 = None

death$_2$

In [10]:
#ysum.addinit("_death2")

class ysum(ysum):
    def _death2(self):

        # this list has an entry for each year after and including the maximum citations ever received (the last time)
        # look ahead to the next ten years and take the average
        next_year_sums = [
            (ycheck, self.sum_between(ycheck, ycheck+9))
            for ycheck in range(self.maxcounty, RELIABLE_DATA_ENDS_HERE - 10) 
        ]

        # need to make sure ALL subsequent decade intervals are also less...
        my_death_year = None

        l = len(next_year_sums)
        for i in range(l):
            not_this_one = False
            for j in range(i,l):
                if next_year_sums[j][1] >= self.maxcount:
                    not_this_one = True
                if not_this_one:
                    break

            if not_this_one:
                continue

            my_death_year = next_year_sums[i][0]
            break

        if not len(next_year_sums):
            self.death2 = None
        else:
            self.death2 = my_death_year

        return None
    

In [11]:
#ysum.addinit("_death3")

class ysum(ysum):
    def _death3(self):
        # DEATH3 is last, as long as it's before RELIABLE_DATA_ENDS_HERE - 10
        # no citations in the last 10 years
        self.death3 = None
        if self.last <= RELIABLE_DATA_ENDS_HERE - 10:
            self.death3 = self.last

In [12]:
#ysum.addinit("_death5")

class ysum(ysum):
    def _death5(self):


        # DEATH5
        # 90% of their citations were received before death4, and it's been at least 30% of their lifespan
        myspan = np.array( [self.cits[ comb(dtype, 'fy')][ make_cross({dtype:self.name, 'fy':ycheck}) ] for ycheck in range(1900, 2020)] )

        self.death5 = None

        Ea = np.sum(myspan)
        csum = np.sum(myspan)

        nonzeroyears = list(np.where(myspan>0))
        if not len(nonzeroyears):
            self.death5 = None
            return

        try:
            firsti = np.min(nonzeroyears)
        except:
            print("some strange error, that shouldn't happen, right??")
            return

        first_year = firsti + 1900

        for cci, cc in enumerate(myspan[firsti:]):

            this_year = first_year+cci

            # running residual... 
            Ea -= cc

            # don't let them die too soon
            if cc == 0:
                continue

            if Ea/csum < 0.1 and (RELIABLE_DATA_ENDS_HERE - this_year)/(RELIABLE_DATA_ENDS_HERE - first_year) > 0.3:
                self.death5 = this_year
                break


In [13]:
ysum.addinit("_deathN")

class ysum(ysum):
    def _deathN(self):

        
        def getname(CUTOFF):
            if CUTOFF == 1:
                return 'death_max'
            if CUTOFF == 0:
                return 'death_last'
            
            return "death_%d"%(CUTOFF*10)
        

        # DEATH5
        # 90% of their citations/year were received before death4, and it's been at least 30% of their lifespan
        
        
        # preset all deaths as None
        for CUTOFF in np.linspace(0,1,11):
            setattr(self, getname(CUTOFF), None)

        
        for y in range(self.maxcounty + 1, RELIABLE_DATA_ENDS_HERE):            
            #if (RELIABLE_DATA_ENDS_HERE - y)/(RELIABLE_DATA_ENDS_HERE - self.first) < 0.3:
            #    # eventually we just can't tell anymore...
            #    continue
            if RELIABLE_DATA_ENDS_HERE - y <= 10:
                continue
                
            before = self.sum_between(self.first, y) / (y - self.first)
            after = self.sum_between(y,RELIABLE_DATA_ENDS_HERE) / (RELIABLE_DATA_ENDS_HERE-y)
            
            
            for CUTOFF in np.linspace(0,1,11):
                myname = getname(CUTOFF)
                    
                if getattr(self, myname) is not None:
                    continue
                    
                if after/before <= CUTOFF:
                    setattr(self, myname, y)

# master loop

In [14]:
print("Processing database '%s'"%database_name)

mysums = ysums()
for ci,item in enumerate(ysum.cy):
    
    if ci % 5000 == 0:
        print("Item %s" % ci)
        
    #if ci > 10000:
    #    break
        
    res = {}
    mysums.meta['at least one citation'] += 1

    mysum = ysum( name=item )

    #small error catch
    if hasattr(mysum,'pub') and mysum.first < mysum.pub:
        mysums.meta['first citation before pub date'] += 1
        continue

    # don't care about those with only a single citation
    if mysum.total <= 1:
        mysums.meta['literally 1 citation. dropped.'] += 1
        continue

    # we really don't care about those that never rise in use
    if mysum.first == mysum.maxpropy:
        mysums.meta['never rise'] += 1
        continue
        
    mysums.meta['passed tests pre-blacklist'] += 1

    
    mysums.add(mysum)

Processing database 'sociology-wos'
Item 0
Item 5000
Item 10000
Item 15000
Item 20000
Item 25000
Item 30000
Item 35000
Item 40000
Item 45000
Item 50000
Item 55000
Item 60000
Item 65000
Item 70000
Item 75000
Item 80000
Item 85000
Item 90000
Item 95000
Item 100000
Item 105000
Item 110000
Item 115000
Item 120000
Item 125000
Item 130000
Item 135000
Item 140000
Item 145000
Item 150000
Item 155000
Item 160000
Item 165000
Item 170000
Item 175000
Item 180000
Item 185000
Item 190000
Item 195000
Item 200000
Item 205000
Item 210000
Item 215000
Item 220000
Item 225000
Item 230000
Item 235000
Item 240000
Item 245000
Item 250000
Item 255000
Item 260000
Item 265000
Item 270000
Item 275000
Item 280000
Item 285000
Item 290000
Item 295000


# Debugging goes here

In [16]:
for ys in mysums.ysums[:10]:
    dd = ys.dump()
    for k in sorted(dd):
        print(k, dd[k])
        
    print("----------------------------")

death_1 None
death_2 None
death_3 None
death_4 None
death_5 None
death_6 None
death_7 None
death_8 None
death_9 None
death_last None
death_max None
first 1971
last 2015
maxcount 4
maxcounty 2015
maxprop 0.004449388209121246
maxpropy 2015
name ('across', 'societies')
total 56
totalprop 0.060170277562862175
----------------------------
death_1 None
death_2 None
death_3 None
death_4 None
death_5 None
death_6 None
death_7 None
death_8 None
death_9 None
death_last None
death_max None
first 1970
last 2018
maxcount 36
maxcounty 2007
maxprop 0.058823529411764705
maxpropy 2018
name ('health', 'care')
total 767
totalprop 0.9028539363539622
----------------------------
death_1 None
death_2 None
death_3 None
death_4 None
death_5 None
death_6 None
death_7 None
death_8 None
death_9 None
death_last None
death_max None
first 1965
last 2017
maxcount 34
maxcounty 2014
maxprop 0.03508771929824561
maxpropy 2014
name ('social', 'status')
total 796
totalprop 0.950553535651632
----------------------------
de

In [15]:
print(mysums.meta)

defaultdict(<class 'int'>, {'at least one citation': 296716, 'passed tests pre-blacklist': 97513, 'never rise': 167261, 'literally 1 citation. dropped.': 31671, 'first citation before pub date': 271})


In [16]:
if False:
    mysum_final = pd.DataFrame.from_records( list(x.dump() for x in mysums.ysums) )
    mysum_final.shape
    
if True:
    mysum_final = {}
    for x in mysums.ysums:
        dp = x.dump()
        mysum_final[dp['name']] = dp

In [17]:
varname = "%s.%s.ysum"%(database_name,dtype)
save_variable(varname, mysum_final)

# only necessary if you plan on filtering based on this set

In [20]:
if False:
    save_variable("%s.included_citations"%database_name, set(ysum.keys()))